In [71]:
import torch
from torch import nn
import tensorflow as tf
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter
from tensorboard import notebook

In [72]:
# Load the MNIST data from Keras (Because Keras is a easier to load dataset)
tf.keras.datasets.mnist.load_data(path='mnist.npz')
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

In [73]:
def encoder(batch, label, class_num=10, device='cuda') -> torch.tensor:
    # Create a matrix that contains one-hots vectors
    new_label = torch.zeros(batch, class_num, device=device)
    # Transfer zero to one to shown label in one-hot vector
    for i, l in enumerate(label):
        new_label[i][l - 1] = 1.0
    # Return value
    return new_label


In [74]:
class Mnist(Dataset):
    # Constructor
    def __init__(self, data, label):
        # Initialize the dataset
        self.data = data
        self.label = label

    def __getitem__(self, index):
        # Return the data and label one by one when class is called
        data = torch.tensor(self.data[index])
        # Normalize the image to [0, 1]
        data = data / 255.0
        label = self.label[index]
        return data, label

    def __len__(self):
        # Find and return the length of the dataset
        return len(self.data)

In [ ]:
# Create a dataset class object for training and testing
Train = Mnist(x_train, y_train)
Test = Mnist(x_test, y_test)
# Set the batch size
bs = 1000
# Set the number of epochs
n_epoch = 500

In [ ]:
# Create a dataloader for training, set batch size and shuffle process on this class
TrainLoader = DataLoader(dataset=Train,
                         batch_size=bs,
                         shuffle=True,
                         num_workers=0,
                         pin_memory=False)

# Create a dataloader for testing, set batch size and shuffle process on this class
TestLoader = DataLoader(dataset=Test,
                         batch_size=bs,
                         shuffle=True,
                         num_workers=0,
                         pin_memory=False)

In [76]:
class ConvolutionalModel(nn.Module):
    def __init__(self):
        super(ConvolutionalModel, self).__init__()
        self.feature = torch.nn.Sequential(
            torch.nn.Conv2d(1, 10, kernel_size=5),
            torch.nn.ReLU(inplace=True),
            torch.nn.Conv2d(10, 20, kernel_size=5),
            torch.nn.ReLU(inplace=True)
        )
        self.classify = torch.nn.Sequential(
            torch.nn.Linear(8000, 2000),
            torch.nn.ReLU(inplace=True),
            torch.nn.Linear(2000, 10)
        )
         
    def forward(self, x):
        x = self.feature(x)
        x = x.view(-1, 1, 8000)
        x = self.classify(x)
        return x


In [77]:
# Check GPU availability, if not, use CPU
if torch.cuda.is_available():
    device = "cuda"
    # Set worker numbers to load data from hard(RAM) to GPU
    num_worker = 1
    pin_memory = True
else:
    device = "cpu"
    # Do not use GPU, do not need to worker for loading data on GPU
    num_worker = 0
    pin_memory = False

In [78]:
# Create a model object
CNN = ConvolutionalModel().to(device)

In [83]:
# Monitor the training process with tensorboard, use for plotting the loss and accuracy
writer = SummaryWriter()
# Create a loss function
criterion = torch.nn.CrossEntropyLoss()
# Create a optimizer
optimizer = torch.optim.Adam(CNN.parameters(), lr=0.001, weight_decay=0.0001)
# Create a scheduler to update learning rate
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.1)

In [84]:
def train(model, epoch) -> None:
    # Set the model to training mode
    model.train()
    # Set the loss and accuracy buffer to zero
    run_train_loss = 0.0
    loss = 0.0
    # Iterate over the training dataset
    for data, label in TrainLoader:
        # Delete redundant dimension
        data = torch.unsqueeze(data, 1)

        # Transfer data and label to GPU (labels converted to one-hot vector)
        data = data.to(device)
        label = encoder(data.shape[0], label, 10, device)
        # Calculate the output of the model
        output = model(data)
        # Fix dimension of the output and label
        output_s = torch.squeeze(output, dim=1)
        # Calculate the loss
        loss = criterion(output_s, label)
        # Hold the loss for next iteration
        run_train_loss += loss.item()
        # Optimize values set to zero
        optimizer.zero_grad()
        # Calculate the gradient of the loss with respect to the weights of the model and update the weights with back propagation
        loss.backward()
        # Apply the optimizer to update the weights
        optimizer.step()

    # Calculate the average loss and monitor this value with tensorboard
    writer.add_scalar('Train/Loss', loss / len(TrainLoader), epoch)

In [85]:
def test(model, epoch) -> None:
    # Set the model to evaluation mode
    model.eval()
    # Set the loss and accuracy buffer to zero
    correct = 0
    # Deactivate autograd for evaluation
    with torch.no_grad():
        # Iterate over the testing dataset
        for data, label in TestLoader:
            # Delete redundant dimension
            data = torch.unsqueeze(data, 1)
            # Transfer data and label to GPU
            data = data.to(device)
            label = label.to(device)
            # Calculate the output of the model
            output = model(data)
            # Apply the logistic function to the output
            output = torch.exp(output)
            # Calculate the top-1 accuracy
            output = torch.tensor([torch.argmax(o) for o in output]).to(device)
            predicts = torch.where(output == label)
            # Count the number of correct predictions
            correct += len(predicts[0])

        # Calculate the accuracy with respect to the number of testing samples and monitor this value with tensorboard
        writer.add_scalar('Test/Accuracy', correct / len(TestLoader), epoch)

In [87]:
# Training loop for the model with the specified number of epochs
for epoch in tqdm(range(1, n_epoch + 1)):
    # Train the model
    train(CNN, epoch)
    # Test the model
    test(CNN, epoch)
    # Update the learning rate with the scheduler if the epoch is a multiple of 50
    scheduler.step()

# Fluent print the accuracy of the model
writer.flush()

  7%|▋         | 36/500 [08:24<1:48:19, 14.01s/it]


KeyboardInterrupt: 

## Run "tensorboard --logdir=runs" in the terminal to see the results if you have installed tensorboard, else install it from [here](https://www.tensorflow.org/install/). Also you can run this commands before start the training in jupyter notebook.

In [88]:
# List the available tensoboard runs
notebook.list()
# Connect to the tensoboard server
notebook.display(port=6006, height=1000)

Known TensorBoard instances:
  - port 6006: logdir runs (started 0:37:39 ago; pid 138539)
Selecting TensorBoard with logdir runs (started 0:37:39 ago; port 6006, pid 138539).
